In [1]:

#libraries
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
nltk.download('wordnet')
import string
import re
from nltk.stem import PorterStemmer



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sepehr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sepehr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#read dataset 
data = pd.read_csv(r'C:\Users\sepehr\Desktop\presentation\1974_3493_compressed_articles1.csv\articles1.csv')
data.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [14]:
len(data)

50000

In [22]:
data.at[0,'content']

'WASHINGTON — Congressional Republicans have a new fear when it es to their health care lawsuit against the Obama administration: They might win. The ining Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for and Americans, handing House Republicans a big victory on issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that oute, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been demanding an end to th

In [3]:
import re

data['content'] = data['content'].map(lambda x: re.sub('https|com|tweet|twitter', '', x))


data['paper_text_processed'] = data['content'].map(lambda x: re.sub('[,\.!?—”]', '', x))



print("preprocess 1 done ")
# Convert the titles to lowercase
data['paper_text_processed'] = data['paper_text_processed'].map(lambda x: x.lower())

data['paper_text_processed'] = data['paper_text_processed'].map(lambda x: re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', x)) # remove URLs
data['paper_text_processed'] = data['paper_text_processed'].map(lambda x: re.sub('@[^\s]+', 'AT_USER', x))
data['paper_text_processed'] = data['paper_text_processed'].map(lambda x: re.sub(r'#([^\s]+)', r'\1', x))
print("preprocess 2 done ")
#removing all the emojies 

def emoji(text):
    import emoji
    
    allchars = [str for str in text.decode('utf-8')]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.decode('utf-8').split() if not any(i in str for i in emoji_list)])
    return clean_text
 data['content']= data['content'].apply(lambda x: emoji(x.encode('utf8')))



#removing punct

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

data['paper_text_processed'] = data['paper_text_processed'].apply(lambda x: remove_punct(x))
print("preprocess 3 done ")
data['paper_text_processe'] = data['paper_text_processed'].map(lambda x: re.sub('^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))   


preprocess 1 done 
preprocess 2 done 
preprocess 3 done 


In [15]:
data['paper_text_processe'][0]

'washington     congressional republicans have a new fear when it es to their    health care lawsuit against the obama administration they might win the ining trump administration could choose to no longer defend the executive branch against the suit which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   americans handing house republicans a big victory on    issues but a sudden loss of the disputed subsidies could conceivably cause the health care program to implode leaving millions of people without access to health insurance before republicans have prepared a replacement that could lead to chaos in the insurance market and spur a political backlash just as republicans gain full control of the government to stave off that oute republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the obama health care law angering conservative voters who have been demanding an end to 

In [32]:
words = set(nltk.corpus.words.words())
def remove_non_english(text):

    "".join(w for w in nltk.wordpunct_tokenize(text) \

             if w.lower() in words or not w.isalpha())
    return text
data['paper_text_processed']=data['paper_text_processe']
data['paper_text_processed'] = data['paper_text_processed'].apply(lambda x: remove_non_english(x))

In [33]:

def remove_space(text):
    text=' '.join(text.split())
    return text
data['paper_text_processed'] = data['paper_text_processed'].apply(lambda x: remove_space(x))

In [35]:
data['paper_text_processed']

0        washington congressional republicans have a ne...
1        after the bullet shells get counted the blood ...
2        when walt disney’s “bambi” opened in critics p...
3        death may be the great equalizer but it isn’t ...
4        seoul south korea north korea’s leader kim sai...
5        london queen elizabeth ii who has been battlin...
6        beijing president tsai of taiwan sharply criti...
7        danny cahill stood slightly dazed in a blizzar...
8        just how is hillary kerr the founder of a digi...
9        angels are everywhere in the muñiz family’s ap...
10       with donald j trump about to take control of t...
11       thompsons tex can one of the most promising an...
12       west palm beach fla when donald j trump rang i...
13       this article is part of a series aimed at help...
14       it’s the season for family travel and photos a...
15       finally the second avenue subway opened in new...
16       pages into the journal found in dylann s roof’.

In [36]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

data['paper_text_processed'] = data['paper_text_processed'].apply(lambda x: tokenization(x)) 
print("tokenization done ") 
data['paper_text_processed'] = data['paper_text_processed'].apply(lambda x: remove_stopwords(x)) 
print("remove_stopwords done ")
     

data['paper_text_processed']


tokenization done 
remove_stopwords done 


0        [washington, congressional, republicans, new, ...
1        [bullet, shells, get, counted, blood, dries, v...
2        [walt, disney, bambi, opened, critics, praised...
3        [death, may, great, equalizer, necessarily, ev...
4        [seoul, south, korea, north, korea, leader, ki...
5        [london, queen, elizabeth, ii, battling, cold,...
6        [beijing, president, tsai, taiwan, sharply, cr...
7        [danny, cahill, stood, slightly, dazed, blizza...
8        [hillary, kerr, founder, digital, media, pany,...
9        [angels, everywhere, muñiz, family, apartment,...
10       [donald, j, trump, take, control, white, house...
11       [thompsons, tex, one, promising, troubled, tec...
12       [west, palm, beach, fla, donald, j, trump, ran...
13       [article, part, series, aimed, helping, naviga...
14       [season, family, travel, photos, perhaps, enla...
15       [finally, second, avenue, subway, opened, new,...
16       [pages, journal, found, dylann, roof, car, ass.

In [18]:
data['paper_text_processed']

0        [washington, congressional, republicans, new, ...
1        [bullet, shells, get, counted, blood, dries, v...
2        [walt, disney, bambi, opened, critics, praised...
3        [death, may, great, equalizer, necessarily, ev...
4        [seoul, south, korea, north, korea, leader, ki...
5        [london, queen, elizabeth, ii, battling, cold,...
6        [beijing, president, tsai, taiwan, sharply, cr...
7        [danny, cahill, stood, slightly, dazed, blizza...
8        [hillary, kerr, founder, digital, media, pany,...
9        [angels, everywhere, muñiz, family, apartment,...
10       [donald, j, trump, take, control, white, house...
11       [thompsons, tex, one, promising, troubled, tec...
12       [west, palm, beach, fla, donald, j, trump, ran...
13       [article, part, series, aimed, helping, naviga...
14       [season, family, travel, photos, perhaps, enla...
15       [finally, second, avenue, subway, opened, new,...
16       [, pages, journal, found, dylann, roof, car, a.

In [37]:
from nltk.stem.snowball import SnowballStemmer
stemmer = PorterStemmer()

data['stemmed'] = data['paper_text_processed'].apply(lambda x: [stemmer.stem(y) for y in x])
print("stemming is done ")

stemming is done 


In [38]:
data['stemmed']

0        [washington, congression, republican, new, fea...
1        [bullet, shell, get, count, blood, dri, votiv,...
2        [walt, disney, bambi, open, critic, prais, spa...
3        [death, may, great, equal, necessarili, evenha...
4        [seoul, south, korea, north, korea, leader, ki...
5        [london, queen, elizabeth, ii, battl, cold, we...
6        [beij, presid, tsai, taiwan, sharpli, critic, ...
7        [danni, cahil, stood, slightli, daze, blizzard...
8        [hillari, kerr, founder, digit, media, pani, l...
9        [angel, everywher, muñiz, famili, apart, bronx...
10       [donald, j, trump, take, control, white, hous,...
11       [thompson, tex, one, promis, troubl, technolog...
12       [west, palm, beach, fla, donald, j, trump, ran...
13       [articl, part, seri, aim, help, navig, life, o...
14       [season, famili, travel, photo, perhap, enlarg...
15       [final, second, avenu, subway, open, new, york...
16       [page, journal, found, dylann, roof, car, asse.

In [40]:
# Lemmatizer ex: rocks -> rock
from nltk.stem import WordNetLemmatizer 
  
lemmatiz = WordNetLemmatizer()
def lemmatizer(text):
    listof=[]
    for word in text:
      lemi=  lemmatiz.lemmatize(word)
 
            
            
      listof.append(str(lemi))      
    return listof

data['news_lemmatized'] = data['stemmed'].apply(lambda x: lemmatizer(x))


In [44]:
data['news_lemmatized']

0        [washington, congression, republican, new, fea...
1        [bullet, shell, get, count, blood, dri, votiv,...
2        [walt, disney, bambi, open, critic, prais, spa...
3        [death, may, great, equal, necessarili, evenha...
4        [seoul, south, korea, north, korea, leader, ki...
5        [london, queen, elizabeth, ii, battl, cold, we...
6        [beij, presid, tsai, taiwan, sharpli, critic, ...
7        [danni, cahil, stood, slightli, daze, blizzard...
8        [hillari, kerr, founder, digit, medium, pani, ...
9        [angel, everywher, muñiz, famili, apart, bronx...
10       [donald, j, trump, take, control, white, hous,...
11       [thompson, tex, one, promis, troubl, technolog...
12       [west, palm, beach, fla, donald, j, trump, ran...
13       [articl, part, seri, aim, help, navig, life, o...
14       [season, famili, travel, photo, perhap, enlarg...
15       [final, second, avenu, subway, open, new, york...
16       [page, journal, found, dylann, roof, car, asse.

In [45]:

#Creating posting list using Dictioanry 
#

frequency={}

def creatte_postngs_list(data):

    diction={}

    for i in range(len(data)) :
           
        for words in data[i]:
            
            if (words not in frequency):
                frequency[words] = 0
                frequency[words] = frequency[words] + 1
            
            else:
                frequency[words] = frequency[words] + 1  
                
            if words not in diction.keys():
                    diction[words] = set()
                    diction[words].add(i)
            else:
                diction[words].add(i)
    return  diction
            
            
         
diction=creatte_postngs_list(data['news_lemmatized']) 
    
    

In [46]:
frequency

{'washington': 16375,
 'congression': 3699,
 'republican': 35933,
 'new': 63068,
 'fear': 6342,
 'e': 24678,
 'health': 13745,
 'care': 13527,
 'lawsuit': 3433,
 'obama': 32796,
 'administr': 19734,
 'might': 12823,
 'win': 14185,
 'ine': 3827,
 'trump': 150425,
 'could': 38965,
 'choos': 2828,
 'longer': 4571,
 'defend': 7104,
 'execut': 14173,
 'branch': 1563,
 'suit': 2567,
 'challeng': 7499,
 'author': 15657,
 'spend': 6217,
 'billion': 8118,
 'dollar': 4450,
 'insur': 4564,
 'subsidi': 763,
 'american': 40990,
 'hand': 10533,
 'hous': 33359,
 'big': 11458,
 'victori': 5595,
 'issu': 21718,
 'sudden': 789,
 'loss': 4114,
 'disput': 2062,
 'conceiv': 355,
 'caus': 9322,
 'program': 12234,
 'implod': 121,
 'leav': 12072,
 'million': 23198,
 'peopl': 78088,
 'without': 12430,
 'access': 5584,
 'prepar': 5048,
 'replac': 4911,
 'lead': 13743,
 'chao': 1343,
 'market': 9633,
 'spur': 713,
 'polit': 30565,
 'backlash': 932,
 'gain': 3933,
 'full': 6837,
 'control': 11308,
 'govern': 3146

In [69]:
len(diction['problem'])

7597

In [48]:
import pickle
a_file = open(r"C:\Users\sepehr\Desktop\presentation\ir\data_freq.pkl", "wb")
pickle.dump(diction, a_file)
a_file.close()

In [23]:
import pickle
a_file2 = open(r"C:\Users\sepehr\Desktop\presentation\ir\data_freq.pkl", "rb")
postinglist = pickle.load(a_file2)
    

In [25]:
len(postinglist['dollar'])

3154

In [52]:
#tf_idf 
import math
def word_count(words):
    counts = dict()
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts


tfdict=list()
import math

def tffunc(token,frequency):
    tf_dictionary = {}
    counter=word_count(token)
    for word, count in counter.items(): 
            tf_Dictionary[word] =1+math.log10(count)
            
            if word in frequency.keys():
                tf_dictionary[word]=tf_dictionary[word]*math.log10(len(data)/frequency[word])
                
                
                
            
            
    return tf_dictionary   
    
for i in range(len(data)):    
    tfdict.append(tffunc(data['news_lemmatized'][i],frequency))

In [54]:
#save tfidf file 
import pickle
a_file = open(r"C:\Users\sepehr\Desktop\presentation\ir\tfidf.pkl", "wb")
pickle.dump(tfdict, a_file)
a_file.close()

In [26]:
import pickle
a_file2 = open(r"C:\Users\sepehr\Desktop\presentation\ir\tfidf.pkl", "rb")
tfidf = pickle.load(a_file2)
    

In [27]:
tfidf[100]

{'sparkli': 3.2518119729937998,
 'dress': 1.7393010553838428,
 'heat': 1.4084901910053458,
 'tech': 1.1932774969238187,
 'worn': 2.801980017667437,
 'ball': 1.3829996588791011,
 'shoe': 1.700274846024363,
 'drop': 0.9887679896806341,
 'year': -0.2422099256622633,
 'begun': 1.579384229374235,
 'longer': 1.038958783046688,
 'time': -0.12026709743833368,
 'look': 0.5099667001848347,
 'back': 0.23598834760960172,
 'forward': 0.9724873075511892,
 'see': 0.4697770573752203,
 'e': 0.30666004444320005,
 'fashion': 2.9840767488442164,
 'import': 0.6320108064688966,
 'affect': 1.0694584701355656,
 'life': 0.5218954584259425,
 'interest': 0.8214423099169055,
 'ahead': 1.0645935102476511,
 'open': 0.6712007954222761,
 'thought': 0.7341448864526301,
 'watch': 0.9449438260860428,
 'ing': 1.053767400440788,
 'month': 0.5541585954780993,
 'new': -0.1970639671394392,
 'washington': 0.6307246346283472,
 'pari': 1.4999849830875545,
 'pantsuit': 2.899629454882437,
 'least': 0.6374044424511802,
 'woman': 0

In [57]:
count=10
dictionary_word={}
def championlist(list_data,data,keys):
    dict_champ={}
    for i in range(len(list_data)):
        
        counter=word_count(data['news_lemmatized'][list_data[i]])
        if keys in counter.keys():
            dict_champ[list_data[i]]=counter[keys]
    return   dict_champ      
 
    
    
    
    
dict_cham={}    
for keys in diction.keys():
   r=count 

   dict1=championlist(list(diction[keys]),data,keys)
   
   sort_orders = sorted(dict1.items(), key=lambda x: x[1], reverse=True) 
   if(len(diction[keys])<count):
     r=len(diction[keys])

   dict_cham[keys] =sort_orders[0:r]
                

          

In [64]:
dict_cham["washington"]

[(5659, 32),
 (15664, 26),
 (13282, 24),
 (932, 23),
 (1159, 23),
 (18680, 22),
 (18880, 22),
 (24938, 21),
 (2793, 19),
 (23257, 19)]

In [61]:
dict_cham_final={}
for keys in dict_cham.keys():
    listof=[]
    for i in range(len(dict_cham[keys])):
        
        listof.append(dict_cham[keys][i][0])
    dict_cham_final[keys]  =  listof

In [62]:
dict_cham_final

{'washington': [5659,
  15664,
  13282,
  932,
  1159,
  18680,
  18880,
  24938,
  2793,
  23257],
 'congression': [14120,
  35445,
  14154,
  22799,
  9343,
  36885,
  31199,
  17282,
  14188,
  32592],
 'republican': [3036,
  1115,
  22223,
  17533,
  2069,
  18159,
  2808,
  13682,
  3323,
  15465],
 'new': [29114, 30733, 23425, 21356, 17533, 26451, 6605, 49997, 15664, 3172],
 'fear': [19901, 34329, 11060, 5937, 47113, 18250, 33256, 2967, 36259, 12663],
 'e': [6819, 1165, 17533, 6912, 5718, 25008, 38236, 6242, 49910, 2449],
 'health': [48382, 20563, 1115, 40161, 15933, 22621, 12680, 18067, 513, 5429],
 'care': [1115, 513, 19407, 5877, 12680, 2081, 3664, 11373, 1622, 32517],
 'lawsuit': [32328, 1661, 5008, 40965, 563, 34359, 34501, 3694, 5451, 17398],
 'obama': [26419, 31813, 2669, 17548, 17533, 30092, 325, 33835, 13282, 5861],
 'administr': [49992,
  8894,
  34304,
  32856,
  13360,
  49841,
  33895,
  3578,
  39606,
  34096],
 'might': [6407, 16857, 19822, 21527, 7174, 24566, 2938

In [63]:
import pickle
a_file = open(r"C:\Users\sepehr\Desktop\presentation\ir\data_cham_final.pkl", "wb")
pickle.dump(dict_cham_final, a_file)
a_file.close()